In [5]:
from tinydb import TinyDB, where
from matplotlib import pyplot

import sys
sys.path.append('../tasks')
from data import ALL_REGRESSION_DATATSETS
regression_datasets = list(ALL_REGRESSION_DATATSETS.keys())
regression_datasets.sort()


['boston', 'concrete', 'energy', 'kin8nm', 'naval', 'power', 'protein', 'winered', 'winewhite', 'yacht']


In [14]:
db = TinyDB('db.json')

model = 'linear' #'variationally_sparse_gp' #'linear'
for dataset in regression_datasets:
    
    print(dataset)
    res = db.search((where('model')==model) & (where('dataset')==dataset))
    if len(res)>0:
        print([r['test_likelihood'] for r in res])

boston
[-0.5445419727270256, -0.5445419727270256, -0.5445419727270256, -0.5445419727270256, -0.5445419727270256]
concrete
[-0.9650463081954531, -0.9650463081954531, -0.9650463081954531, -0.9650463081954531, -0.9650463081954531]
energy
[-0.06190911002124293, -0.06190911002124293, -0.06190910333798008, -0.06190911002124293, -0.06190910333798008]
kin8nm
[-1.1178051241792488, -1.1178051241792488, -1.1178051241792488, -1.1178051241792488, -1.1178051241792488]
naval
[-0.4542847275722604, -0.4542847275722604, -0.4542847275722604, -0.4542847275722604, -0.4542847275722604]
power
[-0.1403387119695741, -0.1403387119695741, -0.1403387119695741, -0.1403387119695741, -0.1403387119695741]
protein
[-1.2479220263180402, -1.2479220263180402, -1.2479220263180402, -1.2479220263180402, -1.2479220263180402, -1.2479220263180402, -1.2479220263180402, -1.2479220263180402, -1.2479220263180402]
winered
[-1.1342291894320327, -1.1342291894320327, -1.1342291894320327, -1.1342291894320327, -1.1342291894320327]
winew